In [1]:
if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    boo_pythonNB_environment = True
else:
    boo_pythonNB_environment = False

print('Execution Environment: ' + get_ipython().__class__.__name__ + '; setting boo_pythonNB_environment to: ' \
      + str(boo_pythonNB_environment))

Execution Environment: ZMQInteractiveShell; setting boo_pythonNB_environment to: True


In [2]:
from gssutils import *

#if boo_pythonNB_environment == False:
scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/' \
                  'healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk')
scraper

#else:
#    tabs = loadxlstabs('/Users/martyn/Downloads/hslepivotab1.xlsx') # Not significantly quicker with local read.
# Download not the bottle-neck of the 50Mb file - read to memory is what is taking the time.    


## Health state life expectancy - all ages, UK

Pivot tables for health state life expectancy by sex and area type, divided by two-year intervals starting from 2009 to 2011.

### Description

Pivot tables for health state life expectancy by sex and area type, divided by two-year intervals starting from 2009 to 2011.

### Distributions

1. Health state life expectancy - all ages, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/hslepivotab1.xlsx)) - 2016-11-29
1. Health state life expectancy - all ages, UK ([application/zip](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v1/healthexpectanciespivottables.zip)) - 2016-11-29
1. Health state life expectancy - all ages, UK ([application/zip](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v2/healthexpectanciespivottablesversion2.zip)) - 2016-11-29
1. Health state life expectancy - all ages, UK ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/datasets/healthstatelifeexpectancyallagesuk/current/previous/v3/refpivottablesfinal.xlsx)) - 2016-11-29


In [3]:
tabs = scraper.distributions[0].as_databaker()
distribution = scraper.distributions[0]
display(distribution)

Spreadsheet is 50Mb - note to developers - should we cut down the spreadsheet by deleting hidden columns etc. before processing? Each TAB is also calculated from pivot(s); the RAW data being hidden.

In [4]:
str_tabsheetsinfocus = 'H'
if boo_pythonNB_environment == True:
    print(str_tabsheetsinfocus)

H


In [82]:
i = 0
pd_df_name = []
pd_df_cleaned = []

for tab in tabs:
    if tabs[i].name.startswith(str_tabsheetsinfocus):
        
        print(' Scanning: ' + tabs[i].name)

        if tabs[i].name == 'HE - Country level estimates':
            try:
                pd_df_name.append(tabs[i].name)
                pd_tab = distribution.as_pandas(sheet_name = tabs[i].name) #, skiprows=1, header=None)
                pd_df_dimensions = pd_tab.iloc[:, :7]
                pd_df_observations = pd_tab.iloc[:, 7:18]
                pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                pd_df_cleaned.append(pd_df_original.dropna(how='all'))
                print('Processed: ' + tabs[i].name)
                #break
            except ERR_HECountryLevelEstimates:
                print('Error within ' + str(pd_df_name.append(tabs[i].name)) + ' process to extract to Pandas DF.')
            
        if tabs[i].name == 'HE - Region level estimates':
            try:
                pd_df_name.append(tabs[i].name)
                pd_tab = distribution.as_pandas(sheet_name = tabs[i].name)
                pd_df_dimensions = pd_tab.iloc[:, :8]
                pd_df_observations = pd_tab.iloc[:, 8:19]
                pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                pd_df_cleaned.append(pd_df_original.dropna(how='all'))
                print('Processed: ' + tabs[i].name)
            except ERR_HERegionLevelEstimates:
                print('Error within ' + str(pd_df_name.append(tabs[i].name)) + ' process to extract to Pandas DF.')

                
        if tabs[i].name == 'HE - MC,CA,WHB Estimates':
            try:
                pd_df_name.append(tabs[i].name)
                pd_tab = distribution.as_pandas(sheet_name = tabs[i].name)
                pd_df_dimensions = pd_tab.iloc[:, :7]
                pd_df_observations = pd_tab.iloc[:, 7:18]
                pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                pd_df_cleaned.append(pd_df_original.dropna(how='all'))            
                print('Processed: ' + tabs[i].name)
            except ERR_HEMCMAWHBLevelEstimates:
                print('Error within ' + str(pd_df_name.append(tabs[i].name)) + ' process to extract to Pandas DF.')

            
        if tabs[i].name == 'HE  - Local area estimates': #Notice additional spaces that may be removed in future...
            try:
                pd_df_name.append(tabs[i].name)
                pd_tab = distribution.as_pandas(sheet_name = tabs[i].name)
                pd_df_dimensions = pd_tab.iloc[:, :8]
                pd_df_observations = pd_tab.iloc[:, 8:19]
                pd_df_original = pd.concat([pd_df_dimensions, pd_df_observations], axis=1, sort=False)
                pd_df_cleaned.append(pd_df_original.dropna(how='all'))
                print('Processed: ' + tabs[i].name)
            except ERR_HERegionLevelEstimates:
                print('Error within ' + str(pd_df_name.append(tabs[i].name)) + ' process to extract to Pandas DF.')
            
            
    i += 1

 Scanning: HE - Country level estimates
Processed: HE - Country level estimates
 Scanning: HE - Region level estimates
Processed: HE - Region level estimates
 Scanning: HE - MC,CA,WHB Estimates
Processed: HE - MC,CA,WHB Estimates
 Scanning: HE  - Local area estimates
Processed: HE  - Local area estimates


In [83]:
#print(list(pd_df_dimensions.columns))
#print(list(pd_df_observations.columns))

In [94]:
#This code is for testing purposes - not for final.
for i in range(len(pd_df_name)):
    print(pd_df_name[i])
    
#print(pd_df_name[0])
#pd_df_cleaned[0].head(3)

HE - Country level estimates
HE - Region level estimates
HE - MC,CA,WHB Estimates
HE  - Local area estimates


In [95]:
#This code is for testing purposes - not for final.
pd_df_cleaned[0].head(3)

,Period_x2,Code,Country,sex1,Gender,ageband,age group,Life Expectancy (LE)_,LE Lower CI_,LE Upper CI_,Healthy Life Expectancy (HLE) _,HLE Lower CI_,HLE Upper CI_,"Proportion of Life Spent in ""Good"" Health (%)_",Disability Free Life Expectancy (DFLE)_,DFLE Lower CI_,DFLE Upper CI_,Proportion of Life Spent Disability Free (%)_
0,2009-11,E92000001,England,1.0,Male,1.0,<1,78.78073,78.75026,78.81120,63.02647,62.87787,63.17508,80.00240,63.49672,63.35193,63.64150,80.59930
1,2009-11,E92000001,England,1.0,Male,2.0,01-04,78.17075,78.14207,78.19942,62.37935,62.23008,62.52862,79.79884,62.85968,62.71425,63.00510,80.41330
2,2009-11,E92000001,England,1.0,Male,3.0,05-09,74.22782,74.19942,74.25623,58.59795,58.44976,58.74614,78.94338,59.10273,58.95841,59.24704,79.62342


In [96]:
#This code is for testing purposes - not for final.
#Code may be used where renaming is required...
col_dict = {
    'Sex': 'Gender',
    'Period': 'Period_x2'
}

for i in range(len(pd_df_cleaned)):
    #pd_df_cleaned[i].rename(columns={'Sex': 'Gender', 'Period': '2Yearsomethingorother'}, inplace=True)
    # Painfully slow using .rename function...
    pd_df_cleaned[i].columns = [col_dict.get(x, x) for x in pd_df_cleaned[i].columns]


In [97]:
#This code is for testing purposes - not for final.
pd_df_cleaned[0].head(3)

,Period_x2,Code,Country,sex1,Gender,ageband,age group,Life Expectancy (LE)_,LE Lower CI_,LE Upper CI_,Healthy Life Expectancy (HLE) _,HLE Lower CI_,HLE Upper CI_,"Proportion of Life Spent in ""Good"" Health (%)_",Disability Free Life Expectancy (DFLE)_,DFLE Lower CI_,DFLE Upper CI_,Proportion of Life Spent Disability Free (%)_
0,2009-11,E92000001,England,1.0,Male,1.0,<1,78.78073,78.75026,78.81120,63.02647,62.87787,63.17508,80.00240,63.49672,63.35193,63.64150,80.59930
1,2009-11,E92000001,England,1.0,Male,2.0,01-04,78.17075,78.14207,78.19942,62.37935,62.23008,62.52862,79.79884,62.85968,62.71425,63.00510,80.41330
2,2009-11,E92000001,England,1.0,Male,3.0,05-09,74.22782,74.19942,74.25623,58.59795,58.44976,58.74614,78.94338,59.10273,58.95841,59.24704,79.62342


In [ ]:
#col_dict = {'gdp': 'log(gdp)', 'cap': 'cap_mod'}   ## key→old name, value→new name
#df.columns = [col_dict.get(x, x) for x in df.columns]

In [98]:
#This code is for testing purposes - not for final.
#Will need a basic loop through of partitions in final code...
pd_df_cleaned[0].head(3).to_csv(index=False)

'Period_x2,Code,Country,sex1,Gender,ageband,age group,Life Expectancy (LE)_,LE Lower CI_,LE Upper CI_,Healthy Life Expectancy (HLE) _,HLE Lower CI_,HLE Upper CI_,"Proportion of Life Spent in ""Good"" Health (%)_",Disability Free Life Expectancy (DFLE)_,DFLE Lower CI_,DFLE Upper CI_,Proportion of Life Spent Disability Free (%)_\n2009-11,E92000001,England,1.0,Male,1.0,<1,78.78073,78.75026,78.8112,63.02647,62.87787,63.17508,80.0024,63.49672,63.35193,63.6415,80.5993\n2009-11,E92000001,England,1.0,Male,2.0,01-04,78.17075,78.14207,78.19942,62.37935,62.23008,62.52862,79.79884,62.85968,62.71425,63.0051,80.4133\n2009-11,E92000001,England,1.0,Male,3.0,05-09,74.22782,74.19942,74.25623,58.59795,58.44976,58.74614,78.94338,59.10273,58.95841,59.24704,79.62342\n'